# 1. 환경 설정

In [1]:
%pip install langgraph langchain-core langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

# 2. 그래프 요소 정의
## 1. 상태 작성

In [3]:
from typing import TypedDict, Optional

class AgentState(TypedDict):
    user_input: str               # 사용자의 질문
    draft_answer: Optional[str]   # 현재까지 생성된 답변
    eval_result: Optional[str]    # 평가 결과(예: "satisfied" / "unsatisfied")
    reasoning: Optional[str]      # 평가 과정 설명 (디버깅용, 최종 출력에는 안 써도 됨)
    loop_count: int               # 몇 번째 루프인지
    max_loops: int                # 최대 루프 횟수

## 2. llm 호출 정의

In [ ]:
from langchain_upstage import ChatUpstage
from langchain.schema import SystemMessage, HumanMessage
# from langchain_openai import ChatOpenAI

# 1) 모델 인스턴스를 전역으로 한 번만 만들고 재사용합니다.
llm = ChatUpstage(
    model="solar-mini",
    temperature=0.7,
)

# llm_validate = ChatOpenAI(
#     model="gpt-5-mini"
# )

llm_validate = ChatUpstage(
    model="solar-pro2",
    temperature=0.7
)

def call_llm(prompt: str) -> str:
    """
    Upstage 모델을 한 번 호출하고 string만 돌려줍니다.
    LangGraph 노드 안에서는 이 함수만 부릅니다.
    """
    messages = [
        SystemMessage(content="당신은 정중하고 정확하게 답변하는 전문가 어시스턴트입니다."),
        HumanMessage(content=prompt),
    ]

    # invoke()는 ChatModel 표준 인터페이스입니다.
    result = llm.invoke(messages)

    # result는 보통 AIMessage 객체이며 .content에 텍스트가 들어 있습니다.
    return result.content.strip()

def call_llm_val(prompt: str) -> str:
    """
    Upstage 모델을 한 번 호출하고 string만 돌려줍니다.
    LangGraph 노드 안에서는 이 함수만 부릅니다.
    """
    messages = [
        SystemMessage(content="당신은 정중하고 정확하게 답변하는 전문가 어시스턴트입니다."),
        HumanMessage(content=prompt),
    ]

    # invoke()는 ChatModel 표준 인터페이스입니다.
    result = llm_validate.invoke(messages)

    # result는 보통 AIMessage 객체이며 .content에 텍스트가 들어 있습니다.
    return result.content.strip()

## 3. 노드 정의

In [12]:
# 답변 초안 생성 노드
def generate_answer(state: AgentState) -> AgentState:
    """
    사용자 요청(user_input)을 보고 답변 초안(draft_answer)을 생성합니다.
    loop_count도 함께 고려해서 '다시 시도'인지 아닌지 모델에게 알려줄 수 있습니다.
    """
    user_input = state["user_input"]
    loop_count = state["loop_count"]

    prompt = f"""
당신은 사용자 질문에 답변을 작성하는 어시스턴트입니다.
아래의 사용자 질문에 대해 명확하고 유용한 답변을 제시해 주세요.

[시도 번호]: {loop_count}
[사용자 질문]:
{user_input}

[현재까지의 답변 초안(있으면 참고, 없어도 무시해도 좋음)]:
{state.get("draft_answer") or "(없음)"}
"""
    draft = call_llm(prompt).strip()

    # state 업데이트 후 반환
    new_state = state.copy()
    new_state["draft_answer"] = draft
    return new_state

# 만족할만한 응답인지 검증하는 노드
def evaluate_answer(state: AgentState) -> AgentState:
    """
    draft_answer가 사용자가 만족할 만한지 LLM에게 물어보고,
    'satisfied' 또는 'unsatisfied'로 태깅합니다.
    """
    user_input = state["user_input"]
    draft_answer = state["draft_answer"]

    eval_prompt = f"""
당신은 엄격한 리뷰어입니다.
아래 기준으로만 판단해 주세요:

1. 이 답변이 사용자의 실제 요구를 충족합니까?
2. 논리적으로 틀린 내용이나 위험한 내용은 없습니까?
3. 사용자가 실제로 "이 정도면 됐다"라고 할 법합니까?

출력 형식은 JSON 한 줄로만 주세요:
{{
  "verdict": "satisfied" 또는 "unsatisfied",
  "reason": "왜 그렇게 판단했는지 한두 문장 설명"
}}

[사용자 질문]
{user_input}

[제안된 답변]
{draft_answer}
"""
    raw_eval = call_llm_val(eval_prompt)

    # raw_eval을 파싱해야 합니다.
    # 여기서는 최소 파서(매우 단순)만 작성하겠습니다.
    import json
    verdict = "unsatisfied"
    reason = "parse_failed"
    try:
        parsed = json.loads(raw_eval.replace("```","").strip())
        verdict = parsed.get("verdict", "unsatisfied")
        reason = parsed.get("reason", "")
    except Exception:
        pass

    new_state = state.copy()
    new_state["eval_result"] = verdict
    new_state["reasoning"] = reason
    new_state["loop_count"] = state["loop_count"] + 1
    return new_state

## 4. 조건 분기 함수

In [13]:
def should_loop_or_finish(state: AgentState) -> str:
    """
    LangGraph에서 conditional_edge에 사용됩니다.
    반환값에 따라 다른 edge로 갑니다.
    """
    verdict = state.get("eval_result", "unsatisfied")
    loops  = state.get("loop_count", 1)

    # 1. 루프 한계 먼저 검사합니다.
    #    loop_count는 generate_answer에서 +1씩 증가하므로
    #    loops가 max_loops 이상이면 그냥 끝냅니다.
    print(loops)
    if loops >= state.get("max_loops",5):
        return "finish"
    
    if verdict == "satisfied":
        return "finish"
    else:
        return "retry"

## 5. 그래프 구축

In [14]:
from langgraph.graph import StateGraph, END

# 그래프 객체 생성 (상태 타입 지정)
workflow = StateGraph(AgentState)

# 노드 등록
workflow.add_node("generate_answer", generate_answer)
workflow.add_node("evaluate_answer",  evaluate_answer)

# 시작 노드 지정
workflow.set_entry_point("generate_answer")

# 노드 간 기본 연결
workflow.add_edge("generate_answer", "evaluate_answer")

# evaluate_answer 이후 분기:
# - 만족: 종료(END)
# - 불만족: 다시 generate_answer로
workflow.add_conditional_edges(
    "evaluate_answer",
    should_loop_or_finish,
    {
        "finish": END,
        "retry": "generate_answer",
    },
)

# graph 컴파일
app = workflow.compile()

## 6. 그래프 실행 예시 

In [16]:
# LangGraph는 "스트리밍" 스타일 step 호출도 있고, 한 번에 돌려서 최종 상태를 받을 수도 있습니다.
# 여기서는 while 루프 없이 graph 자체에 맡깁니다.

initial_state: AgentState = {
    "user_input": "심즈에서 심들의 기분을 최대치로 유지하는 가장 효율적인 방법은 뭐야?",
    "draft_answer": None,
    "eval_result": None,
    "reasoning": None,
    "loop_count": 0,
    "max_loops": 5,
}

# app.invoke는 상태를 받아서 끝날 때까지 돌리고 최종 상태를 반환합니다.
final_state = app.invoke(initial_state)

print("----- 최종 답변 후보 -----")
print(final_state["draft_answer"])
print()
print("----- 평가 정보 -----")
print("평가 결과:", final_state["eval_result"])
print("평가 사유:", final_state["reasoning"])
print("총 루프 횟수(추정):", final_state["loop_count"])

1
----- 최종 답변 후보 -----
심즈에서 심들의 기분을 최대치로 유지하는 것은 게임 플레이를 더욱 즐겁고 원활하게 진행하는 데 도움이 됩니다. 다음은 심들의 기분을 최대치로 유지하는 가장 효율적인 방법들입니다:

1. **기본적인 필요 충족**:
   - 배고픔, 목마름, 위생, 에너지, 행복 등 심들의 기본적인 필요를 정기적으로 충족시켜 주세요. 이를 위해 식사와 음료를 제공하고, 위생을 유지하며, 충분한 휴식을 취할 수 있도록 해줍니다.

2. **행복한 환경 조성**:
   - 심들의 성격에 맞는 편안한 가구를 배치하고, 조명과 색상을 통해 쾌적한 분위기를 만들어 주세요. 또한, 심들이 좋아하는 활동을 할 수 있는 공간을 마련하는 것도 중요합니다.

3. **사회적 상호작용**:
   - 심들이 친구나 가족과 상호작용하도록 장려하세요. 친구를 초대하여 파티를 열거나, 가족과 함께 게임을 하며 시간을 보내도록 합니다. 이는 심들의 사회적 필요를 충족시키고 기분을 높여줍니다.

4. **취미와 직업**:
   - 심들에게 취미 활동이나 직업을 부여하여 그들의 목표와 야망을 충족시키도록 합니다. 취미나 직업을 통해 심들은 성취감을 느끼고, 이는 기분을 높이는 데 기여합니다.

5. **긍정적인 피드백**:
   - 심들이 긍정적인 성과를 이루거나 좋은 행동을 할 때 칭찬하고 격려해 주세요. 이는 심들의 자신감을 높이고 기분을 개선하는 데 도움이 됩니다.

6. **스트레스 관리**:
   - 심들이 스트레스를 받지 않도록 주의 깊게 관리합니다. 스트레스를 유발할 수 있는 상황(예: 과도한 업무, 가족 갈등)을 피하고, 스트레스를 해소할 수 있는 활동(예: 운동, 음악 듣기)을 장려합니다.

7. **관계 유지**:
   - 심들이 서로 긍정적인 관계를 유지하도록 돕습니다. 사랑스러운 관계를 유지하거나, 친구와의 관계를 강화하면 심들의 기분이 더욱 높아질 수 있습니다.

이러한 방법들을 통해 심들의 기분을 최대치로 유지하고, 게임 내에서 더욱 풍성한 경